In [1]:
import pandas as pd

C:\Users\prith\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#Modify this at your own disgression
initial_claims = pd.read_csv("InitialClaims.csv",  index_col='DATE')
mortgage_30US = pd.read_csv("MORTGAGE30US.csv",   index_col='observation_date')
national_HPI = pd.read_excel("hpi_po_monthly_hist.xlsx", sheet_name='HPI_PO_monthly_hist', skiprows=3)
state_HPI = pd.read_excel("hpi_po_state.xlsx")
unrate = pd.read_csv("UNRATE.csv", index_col='observation_date')

In [3]:
#initial_claims[(initial_claims.index > '2023-03-20') & (initial_claims.index < '2023-04-30')]
initial_claims.index = pd.to_datetime(initial_claims.index)

claims_m = initial_claims[['ICSA']].resample('ME').mean()          # monthly avg

In [4]:
mortgage_30US.index = pd.to_datetime(mortgage_30US.index)
mort_m = mortgage_30US[['MORTGAGE30US']].resample('ME').mean().rename(columns ={ 'MORTGAGE30US' : 'mort30_m'})

In [5]:
#national_HPI
national_HPI = national_HPI.set_index('Month')
national_HPI.index = national_HPI.index.to_period("M").to_timestamp("M")
national_HPI.index = pd.to_datetime(national_HPI.index)

national_HPI_m = national_HPI[['USA\n\n(NSA)']]
national_HPI_m = national_HPI_m.rename(columns={'USA\n\n(NSA)': 'HPI_NSA'})

In [6]:
nsa_col = 'index_nsa'
sa_col = 'index_sa'
state_HPI = state_HPI[['state', 'yr', 'qtr'] + [nsa_col,sa_col]].copy()

# Map quarter to months
q_to_months = {1: [1, 2, 3], 2: [4, 5, 6], 3: [7, 8, 9], 4: [10, 11, 12]}
state_HPI["month"] = state_HPI["qtr"].map(q_to_months)

# explode one row per month
state_HPI = state_HPI.explode("month", ignore_index=True)

# build month-end dates
state_HPI["date"] = pd.to_datetime(
    dict(year=state_HPI["yr"].astype(int), month=state_HPI["month"].astype(int), day=1)
).dt.to_period("M").dt.to_timestamp("M")

# final tidy columns
state_HPI = state_HPI.rename(columns={sa_col : "state_hpi_sa", nsa_col : "state_hpi_nsa"})

state_HPI_m = state_HPI.sort_values(["state","date"]).drop(columns= ['yr', 'qtr', 'month'])
state_HPI_m = state_HPI_m.set_index('date')

In [7]:
unrate.index = pd.to_datetime(unrate.index, errors='coerce') 
unrate.index = unrate.index.to_period("M").to_timestamp("M")

In [8]:
# National monthly macro (index by date)
start_date = '2001-01-02'
end_date = '2021-12-30'

claims_m = claims_m[(claims_m.index > start_date) & (claims_m.index < end_date)]
mort_m = mort_m[(mort_m.index > start_date) & (mort_m.index < end_date)]
national_HPI_m = national_HPI_m[(national_HPI_m.index > start_date) & (national_HPI_m.index < end_date)]
unrate = unrate[(unrate.index > start_date) & (unrate.index < end_date)]
state_HPI_m = state_HPI_m[(state_HPI_m.index > start_date) & (state_HPI_m.index < end_date)]

macro_us = pd.concat([claims_m, mort_m], axis=1)
macro_us = macro_us.join(national_HPI_m, how='outer')     # us_hpi_sa, [us_hpi_nsa]
macro_us = macro_us.join(unrate, how='outer')
macro_us = macro_us.join(state_HPI_m, how='outer')

In [9]:
macro_us

,ICSA,mort30_m,HPI_NSA,UNRATE,state,state_hpi_nsa,state_hpi_sa
2001-01-31,340000.0,7.0325,145.68,4.2,AK,140.39,141.07
2001-01-31,340000.0,7.0325,145.68,4.2,AL,144.23,144.02
2001-01-31,340000.0,7.0325,145.68,4.2,AR,142.64,142.91
2001-01-31,340000.0,7.0325,145.68,4.2,AZ,157.26,157.47
2001-01-31,340000.0,7.0325,145.68,4.2,CA,127.81,127.76
...,...,...,...,...,...,...,...
2021-11-30,239500.0,3.0675,359.58,4.2,VT,323.34,326.51
2021-11-30,239500.0,3.0675,359.58,4.2,WA,523.96,530.48
2021-11-30,239500.0,3.0675,359.58,4.2,WI,342.97,346.33
2021-11-30,239500.0,3.0675,359.58,4.2,WV,263.93,263.11


In [10]:
macro_us.to_csv('aggergate_macroeconomic.csv', index=True)